In [ ]:
import os
import time
import requests

import numpy as np 
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry





In [164]:

def parse_team_link(team):
    
    team_home_link = team.find('a',class_='AnchorLink').get('href')
    id_loc = team_home_link.index('id/')
    needed = team_home_link[(id_loc+3):]
    team_id = int(needed[:needed.index('/')])
    team_img = team.find('img')
    full_team_name = team_img['title']
    stats_path = os.path.join('/mens-college-basketball/team/stats/_/id', f'{team_id}')
    roster_path = os.path.join('/mens-college-basketball/team/roster/_/id',f'{team_id}')
    schedule_path = os.path.join('/mens-college-basketball/team/schedule/_/id',f'{team_id}')
    
    return [team_id, full_team_name, team_home_link, schedule_path, roster_path, stats_path]


def get_all_team_ids():
    
    url = 'https://www.espn.com/mens-college-basketball/teams'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    
    team_links = soup.find_all('section', class_='TeamLinks')
    
    team_df = []
    print("Grabbing all team ids...")
    for team_link in tqdm(team_links):
        team_row = parse_team_link(team_link)
        team_df.append(team_row) 
        
    return pd.DataFrame(team_df, columns=['team_id','team_name','team_link','schedule_link','roster_link','stats_link'])


team_ids = get_all_team_ids()

    

Grabbing all team ids...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [00:00<00:00, 27123.08it/s]


In [243]:


DATA_PATH = '../../data/'
team_ids.to_csv(os.path.join(DATA_PATH,'IDs/ESPN/team_ids.csv'), index=False)



In [231]:
team_ids

,team_id,team_name,team_link,schedule_link,roster_link,stats_link
0,91,Bellarmine Knights,/mens-college-basketball/team/_/id/91/bellarmi...,/mens-college-basketball/team/schedule/_/id/91,/mens-college-basketball/team/roster/_/id/91,/mens-college-basketball/team/stats/_/id/91
1,2110,Central Arkansas Bears,/mens-college-basketball/team/_/id/2110/centra...,/mens-college-basketball/team/schedule/_/id/2110,/mens-college-basketball/team/roster/_/id/2110,/mens-college-basketball/team/stats/_/id/2110
2,2198,Eastern Kentucky Colonels,/mens-college-basketball/team/_/id/2198/easter...,/mens-college-basketball/team/schedule/_/id/2198,/mens-college-basketball/team/roster/_/id/2198,/mens-college-basketball/team/stats/_/id/2198
3,526,Florida Gulf Coast Eagles,/mens-college-basketball/team/_/id/526/florida...,/mens-college-basketball/team/schedule/_/id/526,/mens-college-basketball/team/roster/_/id/526,/mens-college-basketball/team/stats/_/id/526
4,294,Jacksonville Dolphins,/mens-college-basketball/team/_/id/294/jackson...,/mens-college-basketball/team/schedule/_/id/294,/mens-college-basketball/team/roster/_/id/294,/mens-college-basketball/team/stats/_/id/294
...,...,...,...,...,...,...
353,2547,Seattle U Redhawks,/mens-college-basketball/team/_/id/2547/seattl...,/mens-college-basketball/team/schedule/_/id/2547,/mens-college-basketball/team/roster/_/id/2547,/mens-college-basketball/team/stats/_/id/2547
354,2617,Stephen F. Austin Lumberjacks,/mens-college-basketball/team/_/id/2617/stephe...,/mens-college-basketball/team/schedule/_/id/2617,/mens-college-basketball/team/roster/_/id/2617,/mens-college-basketball/team/stats/_/id/2617
355,2627,Tarleton Texans,/mens-college-basketball/team/_/id/2627/tarlet...,/mens-college-basketball/team/schedule/_/id/2627,/mens-college-basketball/team/roster/_/id/2627,/mens-college-basketball/team/stats/_/id/2627
356,292,UT Rio Grande Valley Vaqueros,/mens-college-basketball/team/_/id/292/ut-rio-...,/mens-college-basketball/team/schedule/_/id/292,/mens-college-basketball/team/roster/_/id/292,/mens-college-basketball/team/stats/_/id/292


In [ ]:


def scrape_season_schedule_page(team_id, season):
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    
    url = f'https://www.espn.com/mens-college-basketball/team/schedule/_/id/{team_id}/season/{season}'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    
    # check for empty
    no_data_warning = soup.find('div', class_='Schedule__no-data')
    if no_data_warning is not None:
        print("Season does not exist")
        return None
    rows = soup.find_all('tr', class_='Table__TR')

    table = []
    for row in rows:
        spans = row.find_all('span')
        if len(spans) == 0: # header row
            continue
        if spans[0].text == 'DATE': # another header row
            continue

        date = spans[0].text
        home_away = spans[1].text
        opponent = spans[3].text # * means neutral site

        if len(spans) < 5:
            wl_result, score_result, game_link, record = "cancelled","cancelled","cancelled","cancelled"
        else:
            wl_result = spans[4].text
            if len(spans)==5:
                time = wl_result
                wl_result = "upcoming"
                score_result = "upcoming"
                game_link = spans[-1].find('a')['href']
                record = "upcoming"
            else:
                # probably upcoming
                score_result = spans[5].text
                game_link = spans[5].find('a')['href']
                record = spans[6].text

        table.append([date, home_away, opponent, wl_result, score_result, game_link, record])
    table = pd.DataFrame(table, columns=['date','home_away','opponent','wl_result','score_result','game_link','record'])
    
    return table


def pull_all_schedules(tid_df, restart=False):
    
    seasons = range(2003, datetime.now().year)
    
    for index, row in tid_df.iterrows():
        team_id = row['team_id']
        team_schedule_path = os.path.join(DATA_PATH, f'schedules/{team_id}.csv')
        if ((os.path.exists(team_schedule_path))&(not restart)):
            continue
        team_schedule = []
        for season in tqdm(seasons, total=len(seasons)):
            team_season_schedule = scrape_season_schedule_page(team_id, season)
            if team_season_schedule is not None:
                team_season_schedule['season'] = season
                team_schedule.append(team_season_schedule)
            time.sleep(0.75)
        if len(team_schedule)>0:
            team_schedule = pd.concat(team_schedule)
            team_schedule.to_csv(team_schedule_path, index=False)
        
    return

pull_all_schedules(team_ids)



 21%|█████████████████████████████████████▋                                                                                                                                             | 4/19 [00:08<00:30,  2.02s/it]

Season does not exist


 63%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                 | 12/19 [00:20<00:12,  1.85s/it]

Season does not exist


 21%|█████████████████████████████████████▋                                                                                                                                             | 4/19 [00:06<00:24,  1.66s/it]

...

 47%|████████████████████████████████████████████████████████████████████████████████████▊                                                                                              | 9/19 [00:18<00:19,  1.99s/it]

Season does not exist


 32%|████████████████████████████████████████████████████████▌                                                                                                                          | 6/19 [00:15<00:29,  2.30s/it]

Season does not exist


 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                  | 17/19 [00:24<00:02,  1.31s/it]

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<!-- ESPNFITT | 498cc6a2b9c7 | 5400 | a08b3c47371c6651b21736be7452424ab4bbbbf5 | Wed, 09 Feb 2022 18:21:29 GMT -->
<script async="" src="https://dcf.espn.com/TWDC-DTCI/prod/Bootstrap.js" type="text/javascript"></script>
<script type="text/javascript">
        ;(function(){
            function gc(n){n=document.cookie.match("(^|;) ?"+n+"=([^;]*)(;|$)");return n?n[2]:null}function sc(n){document.cookie=n}function smpl(n){n/=100;return!!n&&Math.random()<=n}var _nr=!1,_nrCookie=gc("_nr");null!==_nrCookie?"1"===_nrCookie&&(_nr=!0):smpl(100)?(_nr=!0,sc("_nr=1; path=/")):(_nr=!1,sc("_nr=0; path=/"));;
            _nr && window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true}};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)retu

### below useful for getting today's schedule

In [123]:

date = str(datetime.today().date()).replace('-','')

date = '20220203'
def get_master_schedule(date):
    url = f"https://www.espn.com/mens-college-basketball/schedule/_/{date}/group/50" # group 50 means Division I
    return

url = f"https://www.espn.com/mens-college-basketball/schedule/_/{date}/group/50"
page = requests.get(url)
soup = BeautifulSoup(page.content, "lxml")

# /

In [124]:
soup

<!DOCTYPE html>
<html class="no-icon-fonts" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="x-ua-compatible"/>
<meta content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="origin-when-cross-origin" name="referrer"/>
<link href="https://www.espn.com/mens-college-basketball/schedule" rel="canonical"/>
<title>NCAAM Division I Schedule - NCAAM - ESPN</title>
<meta content="Check out this NCAAM Schedule, sortable by date and including information on game time, network coverage, and more!" name="description"/>
<meta content="NCAAM schedule,NCAAM season schedule" name="news_keywords"/>
<meta content="NCAAM schedule,NCAAM season schedule" name="keywords"/>
<meta content="116656161708917" property="fb:app_id"/>
<meta content="ESPN.com" property="og:site_name"/>
<meta content="https://www.espn.com/mens-college-basketball/schedule" property="og:url"/>
<meta content="NCAAM

In [125]:


def parse_teams(teams):
    
    # name, team profile link, team abbreviation
    try:
        tname_1,tname_2 = teams[0].find('span').text, teams[1].find('span').text
    except:
        print(teams)
        raise ValueError()
    tlink_1, tlink_2 = teams[0]['href'], teams[1]['href']
    talt_name_1, talt_name_2 = teams[0].find('abbr')['title'], teams[1].find('abbr')['title']
    tabbr_1, tabbr_2 = teams[0].find('abbr').text, teams[1].find('abbr').text
    
    return tname_1, tlink_1, talt_name_1, tabbr_1, tname_2, tlink_2, talt_name_2, tabbr_2

def parse_schedule_game_row(g_row):
    
    data_attrs = []
    col_names = []
    # team logic
    teams = g_row.find_all('a', class_='team-name')
    tname_1, tlink_1, talt_name_1, tabbr_1, tname_2, tlink_2, talt_name_2, tabbr_2 = parse_teams(teams)
    logos = g_row.find_all('img', class_='schedule-team-logo')
    team_1_logo,team_2_logo = logos[0]['src'],logos[1]['src']
    
    # game logic
    home_team = g_row.find('td', class_='home').find('a',class_='team-name').find('span').text
    location = g_row.find('td', class_='schedule-location')
    if location is not None:
        location = location.text
    time_col = [td for td in g_row.find_all('td') if td.get('data-date') is not None]
    if len(time_col)==0:
        time = None
        game_link = None
        game_id = None
    else:
        time = time_col[0].get('data-date')
        game_link = time_col[0].find('a').get('href')
        game_id = int(game_link[(game_link.rfind('/')+1):])
    
    data_attrs.extend([home_team, location,  time, game_link, game_id])
    col_names.extend(['home_team','location','time','game_link','game_id'])
    
    data_attrs.extend([tname_1, tlink_1, talt_name_1, tabbr_1, team_1_logo, tname_2, tlink_2, talt_name_2, tabbr_2, team_2_logo])
    col_names.extend(['t1','t1_link','t1_alt','t1_abbr','t1_logo', 't2','t2_link','t2_alt','t2_abbr','t2_logo'])
    
    return data_attrs, col_names



In [126]:
from tqdm import tqdm
game_rows = soup.find_all("tr", class_=["odd", "even"])

day_games = []
for game_row in tqdm(game_rows):
    game_info, col_names = parse_schedule_game_row(game_row)
    day_games.append(game_info)
    
day_games = pd.DataFrame(day_games, columns=col_names)
day_games
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 2629.32it/s]


,home_team,location,time,game_link,game_id,t1,t1_link,t1_alt,t1_abbr,t1_logo,t2,t2_link,t2_alt,t2_abbr,t2_logo
0,San Diego,None,None,None,None,Gonzaga,/mens-college-basketball/team/_/id/2250/gonzag...,Gonzaga Bulldogs,GONZ,https://a.espncdn.com/combiner/i?img=/i/teamlo...,San Diego,/mens-college-basketball/team/_/id/301/san-die...,San Diego Toreros,USD,https://a.espncdn.com/combiner/i?img=/i/teamlo...
1,Arizona,None,None,None,None,UCLA,/mens-college-basketball/team/_/id/26/ucla-bruins,UCLA Bruins,UCLA,https://a.espncdn.com/combiner/i?img=/i/teamlo...,Arizona,/mens-college-basketball/team/_/id/12/arizona-...,Arizona Wildcats,ARIZ,https://a.espncdn.com/combiner/i?img=/i/teamlo...
2,Arizona State,None,None,None,None,USC,/mens-college-basketball/team/_/id/30/usc-trojans,USC Trojans,USC,https://a.espncdn.com/combiner/i?img=/i/teamlo...,Arizona State,/mens-college-basketball/team/_/id/9/arizona-s...,Arizona State Sun Devils,ASU,https://a.espncdn.com/combiner/i?img=/i/teamlo...
3,Bellarmine,None,None,None,None,Liberty,/mens-college-basketball/team/_/id/2335/libert...,Liberty Flames,LIB,https://a.espncdn.com/combiner/i?img=/i/teamlo...,Bellarmine,/mens-college-basketball/team/_/id/91/bellarmi...,Bellarmine Knights,BELL,https://a.espncdn.com/combiner/i?img=/i/teamlo...
4,Ohio,None,None,None,None,Eastern Michigan,/mens-college-basketball/team/_/id/2199/easter...,Eastern Michigan Eagles,EMU,https://a.espncdn.com/combiner/i?img=/i/teamlo...,Ohio,/mens-college-basketball/team/_/id/195/ohio-bo...,Ohio Bobcats,OHIO,https://a.espncdn.com/combiner/i?img=/i/teamlo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Northern Kentucky,None,None,None,None,Oakland,/mens-college-basketball/team/_/id/2473/oaklan...,Oakland Golden Grizzlies,OAK,https://a.espncdn.com/combiner/i?img=/i/teamlo...,Northern Kentucky,/mens-college-basketball/team/_/id/94/northern...,Northern Kentucky Norse,NKU,https://a.espncdn.com/combiner/i?img=/i/teamlo...
75,Wright State,None,None,None,None,Detroit Mercy,/mens-college-basketball/team/_/id/2174/detroi...,Detroit Mercy Titans,DET,https://a.espncdn.com/combiner/i?img=/i/teamlo...,Wright State,/mens-college-basketball/team/_/id/2750/wright...,Wright State Raiders,WRST,https://a.espncdn.com/combiner/i?img=/i/teamlo...
76,UT Arlington,None,None,None,None,Coastal Carolina,/mens-college-basketball/team/_/id/324/coastal...,Coastal Carolina Chanticleers,CCU,https://a.espncdn.com/combiner/i?img=/i/teamlo...,UT Arlington,/mens-college-basketball/team/_/id/250/ut-arli...,UT Arlington Mavericks,UTA,https://a.espncdn.com/combiner/i?img=/i/teamlo...
77,SIU Edwardsville,None,None,None,None,Eastern Illinois,/mens-college-basketball/team/_/id/2197/easter...,Eastern Illinois Panthers,EIU,https://a.espncdn.com/combiner/i?img=/i/teamlo...,SIU Edwardsville,/mens-college-basketball/team/_/id/2565/siu-ed...,SIU Edwardsville Cougars,SIUE,https://a.espncdn.com/combiner/i?img=/i/teamlo...


In [ ]:


# get_master_schedule <- function(date) {
#   ### Error Testing
#   if(is.na(date)) {
#     stop("date is missing with no default")
#   }
  
#   tmp <- try(as.Date(date))
#   if(class(tmp) == "try-error") {
#     stop("Please enter valid date in the form YYYY-MM-DD")
#   }
  
  
#   date_ <- gsub("-", "", as.character(date))
#   url <- paste0("https://www.espn.com/mens-college-basketball/schedule/_/date/", date_)
  
#   z <- XML::readHTMLTable(RCurl::getURL(url))
#   if(length(z) > 1) {
#     schedule <- as.data.frame(z[[1]])[,c(1,2)]
#     completed <- as.data.frame(z[[2]][-1,1:3])
#     names(completed) <- c("away", "home", "result")
#     names(schedule) <- c("away", "home")
#   } else {
#     ### No Games Scheduled
#     if(z[[1]][1,1] == "No games scheduled") {
#       cat("No games on", as.character(date), "\n")
#       return(NULL)
#     }
#     completed <- as.data.frame(z[[1]][,1:3])
#     names(completed) <- c("away", "home", "result")
#     schedule <- NA
#   }
  
#   n_canceled <- sum(grepl("Canceled", completed$result))
#   n_postponed <- sum(grepl("Postponed", completed$result))
#   completed <- dplyr::filter(completed, result != "Canceled", result != "Postponed")
  
#   ### Extract Ranking
#   ranking <- function(team) {
#     rank <- gsub("[^0-9]", "", team)
#     return(ifelse(rank == "", NA, rank))
#   }
  
#   ### Clean Team Name
#   clean <- function(team) {
#     team <- gsub("[#0-9]", "", team)
#     team <- gsub("\\s[A-Z]*-*[A-Z]*$", "", team)
#     team <- gsub("TA&M", "", team)
#     team <- gsub("W&M", "", team)
#     team <- gsub("\\s*$", "", gsub("^\\s*", "", team))
#     return(team)
#   }
  
#   if(any(!is.na(schedule[1]))) {
#     schedule <- dplyr::mutate(schedule,
#                               "away" = as.character(sapply(schedule$away, clean)),
#                               "home" = as.character(sapply(schedule$home, clean)),
#                               "away_rank" = as.numeric(sapply(schedule$away, ranking)),
#                               "home_rank" = as.numeric(sapply(schedule$home, ranking)),
#                               "away_score" = NA,
#                               "home_score" = NA)
#   }
  
#   x <- RCurl::getURL(url)
#   in_progress <- strsplit(x, "/mens-college-basketball/game\\?gameId=")[[1]]
#   in_progress <- suppressWarnings(as.numeric(unname(sapply(in_progress, function(y){ substring(y, 1, 9) }))))
#   in_progress <- in_progress[-1]
#   in_progress <- in_progress[!is.na(in_progress) & !duplicated(in_progress)]
  
#   x <- strsplit(x, "/mens-college-basketball/game/_/gameId/")
#   x <- suppressWarnings(as.numeric(unname(sapply(x, function(y){ substring(y, 1, 9) }))))
#   x <- x[-1]
#   if(date == Sys.Date()) {
#     x <- c(in_progress, x)
#   }
#   x <- x[!is.na(x) & !duplicated(x)]
#   x <- x[1:(length(x) - n_canceled - n_postponed)]
  
#   ### Add in Completed Games
#   find_anchor <- function(team) {
#     cleaned <- clean(team)
#     team <- gsub("[#0-9]", "", team)
#     team <- gsub("\\s*$", "", gsub("^\\s*", "", team))
#     anchor <- unlist(strsplit(team, ""))[-c(1:(nchar(cleaned) + 1))]
#     return(paste0(anchor, collapse = ""))
#   }
  
#   completed <- dplyr::mutate(completed,
#                              "away" = as.character(sapply(away, clean)),
#                              "home" = as.character(sapply(home, clean)),
#                              "result" = as.character(result),
#                              "away_rank" = as.numeric(sapply(completed$away, ranking)),
#                              "home_rank" = as.numeric(sapply(completed$home, ranking)),
#                              "away_anchor" = sapply(completed$away, find_anchor),
#                              "away_score" = NA,
#                              "home_score" = NA)
  
#   winners <- unname(sapply(completed$result, function(y) { gsub("\\s[0-9]*.*", "", y) }))
#   scores <- as.numeric(gsub("[^0-9]", "", gsub("\\(.*\\)", "", unlist(strsplit(completed$result, ",")))))
  
#   if(length(scores) > 0) {
#     winning_scores <- scores[seq(1, length(scores) - 1, 2)]
#     losing_scores <- scores[seq(2, length(scores), 2)]
    
#     index <- sapply(completed$away_anchor, function(y) { y %in% winners })
#     completed$home_score[index] <- losing_scores[index]
#     completed$home_score[!index] <- winning_scores[!index]
#     completed$away_score[!index] <- losing_scores[!index]
#     completed$away_score[index] <- winning_scores[index]
#   }
  
#   if(any(!is.na(schedule[1]))) {
#     schedule <- rbind(schedule, dplyr::select(completed, -away_anchor, -result))
#   }else{
#     schedule <- completed
#   }
  
#   schedule <- dplyr::filter(schedule, !is.na(home), !is.na(away))
#   schedule <- dplyr::mutate(schedule, "game_id" = x[1:nrow(schedule)])
#   schedule <- dplyr::select(schedule, game_id, away, home, away_rank, home_rank, away_score, home_score)
#   schedule$date <- as.Date(date)
  
#   return(schedule)
# }


In [ ]:

def get_team_ids():
    
    
    return




In [ ]:


def get_season_ids():
    
    return




In [1]:

def get_schedule(team):
    
    return


IndentationError: expected an indented block (1938977072.py, line 1)

In [ ]:


#   ### Get Game IDs
#   if(season == current_season) {
#     game_ids <- get_schedule(team) %>%
#       dplyr::filter(date < Sys.Date()) %>%
#       pull(game_id)
#   } else {
#     game_ids <- get_game_ids(team, season)
#   }


#   ### Get PBP Data
#   pbp_season <- get_pbp_game(game_ids, extra_parse)

#   return(pbp_season)
# }


def get_pbp(team, season):
    
    
    return
